In [8]:
import pandas as pd
import numpy as np
import importlib


In [9]:
raw_data_df = pd.read_csv('../data/raw/raw_data.csv')

print(raw_data_df.head())


     user                timestamp  hours               project
0     ned  2019-09-27 00:00:00 UTC    8.0                bizdev
1  robert      09/27/2019 12:00 AM    8.0                bizdev
2     ned   26 сентября 2019 00:00    4.0                bizdev
3     ned  2019-09-26 00:00:00 UTC    1.0  cultureandmanagement
4     ned  2019-09-26 00:00:00 UTC    1.5            project-00


In [10]:
def validate_columns(df):
    required_columns = ['user', 'hours', 'project', 'timestamp']
    if set(required_columns).issubset(df.columns):
        return True, "All required columns are present and properly named"
    else:
        missing_columns = set(required_columns) - set(df.columns)
        return False, f"Missing columns: {', '.join(missing_columns)}"
    
validate_columns_result = validate_columns(raw_data_df)
print(validate_columns_result)

(True, 'All required columns are present and properly named')


In [11]:
def validate_hours(df):
    def is_valid_hour(value):
        if pd.isna(value):
            return False
        try:
            float_value = float(value)
            return 0 < float_value <= 24  # Adjust range as needed
        except ValueError:
            return False

    valid_hours = df['hours'].apply(is_valid_hour)
    invalid_rows = df[~valid_hours].index.tolist()

    if not invalid_rows:
        return True, "All 'hours' values are valid"
    else:
        return False, f"Invalid 'hours' values in rows: {invalid_rows}"
    
validate_hours_result = validate_hours(raw_data_df)
print(validate_hours_result)

(False, "Invalid 'hours' values in rows: [2916, 5005, 17378]")


In [12]:
from datetime import datetime

def validate_timestamp(df):
    def check_timestamp(ts):
        if not isinstance(ts, str):
            return False, f"Invalid type: {type(ts).__name__}"
        
        formats = [
            '%Y-%m-%d %H:%M:%S UTC',
            '%m/%d/%Y %I:%M %p',
            '%d %B %Y %H:%M',
            '%Y-%m-%d %H:%M:%S.%f UTC'
        ]
        
        for fmt in formats:
            try:
                datetime.strptime(ts, fmt)
                return True, "Valid"
            except ValueError:
                continue
        
        russian_months = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня', 
                          'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']
        
        if any(month in ts for month in russian_months):
            return False, "Convertible if changed to English"
        
        return False, "Not convertible"

    results = df['timestamp'].apply(check_timestamp)
    invalid_timestamps = df[~results.apply(lambda x: x[0])]
    
    if invalid_timestamps.empty:
        return True, "All timestamps are in a standard date time format or convertible"
    else:
        invalid_rows = invalid_timestamps.index.tolist()
        conversion_status = [f"Row {idx}: {status[1]}" for idx, status in zip(invalid_rows, results[invalid_timestamps.index])]
        return False, f"Invalid timestamps: {conversion_status}"

validate_timestamp_result = validate_timestamp(raw_data_df)
print(validate_timestamp_result)

(False, "Invalid timestamps: ['Row 2: Convertible if changed to English', 'Row 20: Convertible if changed to English', 'Row 38: Convertible if changed to English', 'Row 50: Convertible if changed to English', 'Row 91: Convertible if changed to English', 'Row 116: Convertible if changed to English', 'Row 120: Convertible if changed to English', 'Row 130: Convertible if changed to English', 'Row 139: Convertible if changed to English', 'Row 140: Convertible if changed to English', 'Row 155: Convertible if changed to English', 'Row 170: Convertible if changed to English', 'Row 171: Convertible if changed to English', 'Row 242: Convertible if changed to English', 'Row 244: Convertible if changed to English', 'Row 253: Convertible if changed to English', 'Row 269: Convertible if changed to English', 'Row 323: Convertible if changed to English', 'Row 329: Convertible if changed to English', 'Row 391: Convertible if changed to English', 'Row 395: Convertible if changed to English', 'Row 407: 

In [13]:
def check_null_values(df):
    null_rows = df[df.isnull().any(axis=1)]
    if null_rows.empty:
        return True, "No missing or null values found"
    else:
        invalid_rows = null_rows.index.tolist()
        return False, f"Missing or null values found in rows: {invalid_rows}"
    
check_null_values_result = check_null_values(raw_data_df)
print(check_null_values_result)

(False, 'Missing or null values found in rows: [15797, 15798, 17572, 17573, 17574]')


In [14]:
validation_results = []

validation_results.append({"validation_type": "Column Presence", "validation_remarks": validate_columns_result[1]})
validation_results.append({"validation_type": "Hours Validation", "validation_remarks": validate_hours_result[1]})
validation_results.append({"validation_type": "Timestamp Format", "validation_remarks": validate_timestamp_result[1]})
validation_results.append({"validation_type": "Null Values Check", "validation_remarks": check_null_values_result[1]})

validation_report = pd.DataFrame(validation_results)
validation_report.head()


,validation_type,validation_remarks
0,Column Presence,All required columns are present and properly ...
1,Hours Validation,"Invalid 'hours' values in rows: [2916, 5005, 1..."
2,Timestamp Format,Invalid timestamps: ['Row 2: Convertible if ch...
3,Null Values Check,"Missing or null values found in rows: [15797, ..."


In [15]:
validation_report.to_csv('../data/reports/validation_report.csv', index=False)
validation_report.to_csv('../data/from_scratch/validation_report.csv', index=False)

In [16]:
raw_data_df = pd.read_csv('../data/raw/raw_data.csv')

def remove_null_values(df):
    """Remove rows with null values from the dataset."""
    return df.dropna()

cleaned_df = remove_null_values(raw_data_df)
cleaned_df.head()

,user,timestamp,hours,project
0,ned,2019-09-27 00:00:00 UTC,8.0,bizdev
1,robert,09/27/2019 12:00 AM,8.0,bizdev
2,ned,26 сентября 2019 00:00,4.0,bizdev
3,ned,2019-09-26 00:00:00 UTC,1.0,cultureandmanagement
4,ned,2019-09-26 00:00:00 UTC,1.5,project-00


In [17]:
def transform_dates(df):
    """Transform convertible dates into standard datetime format in English."""
    def convert_date(date_str):
        russian_months = {
            'января': 'January', 'февраля': 'February', 'марта': 'March',
            'апреля': 'April', 'мая': 'May', 'июня': 'June',
            'июля': 'July', 'августа': 'August', 'сентября': 'September',
            'октября': 'October', 'ноября': 'November', 'декабря': 'December'
        }
        for rus, eng in russian_months.items():
            if rus in date_str:
                return date_str.replace(rus, eng)
        return date_str

    # Create a copy of the DataFrame
    df_copy = df.copy()
    df_copy['timestamp'] = df_copy['timestamp'].apply(convert_date)
    return df_copy

cleaned_df = transform_dates(cleaned_df)
cleaned_df.head()

,user,timestamp,hours,project
0,ned,2019-09-27 00:00:00 UTC,8.0,bizdev
1,robert,09/27/2019 12:00 AM,8.0,bizdev
2,ned,26 September 2019 00:00,4.0,bizdev
3,ned,2019-09-26 00:00:00 UTC,1.0,cultureandmanagement
4,ned,2019-09-26 00:00:00 UTC,1.5,project-00


In [18]:
def remove_duplicates(df):
    """Remove duplicate rows from the dataset."""
    return df.drop_duplicates()

cleaned_df = remove_duplicates(cleaned_df)
cleaned_df.head()


,user,timestamp,hours,project
0,ned,2019-09-27 00:00:00 UTC,8.0,bizdev
1,robert,09/27/2019 12:00 AM,8.0,bizdev
2,ned,26 September 2019 00:00,4.0,bizdev
3,ned,2019-09-26 00:00:00 UTC,1.0,cultureandmanagement
4,ned,2019-09-26 00:00:00 UTC,1.5,project-00


In [19]:
def strip_whitespace(df):
    """Strip whitespace from string fields."""
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].str.strip()
    return df

cleaned_df = strip_whitespace(cleaned_df)
cleaned_df.head()

,user,timestamp,hours,project
0,ned,2019-09-27 00:00:00 UTC,8.0,bizdev
1,robert,09/27/2019 12:00 AM,8.0,bizdev
2,ned,26 September 2019 00:00,4.0,bizdev
3,ned,2019-09-26 00:00:00 UTC,1.0,cultureandmanagement
4,ned,2019-09-26 00:00:00 UTC,1.5,project-00


In [20]:
def convert_timestamps(df):
    """Convert timestamps to a standard format and ensure they are in UTC."""
    def to_utc(ts):
        try:
            dt = pd.to_datetime(ts, utc=True)
            return dt.strftime('%Y-%m-%d %H:%M:%S UTC')
        except:
            return ts  # Return original if conversion fails

    df['timestamp'] = df['timestamp'].apply(to_utc)
    return df

cleaned_df = convert_timestamps(cleaned_df)
cleaned_df.head()


,user,timestamp,hours,project
0,ned,2019-09-27 00:00:00 UTC,8.0,bizdev
1,robert,2019-09-27 00:00:00 UTC,8.0,bizdev
2,ned,2019-09-26 00:00:00 UTC,4.0,bizdev
3,ned,2019-09-26 00:00:00 UTC,1.0,cultureandmanagement
4,ned,2019-09-26 00:00:00 UTC,1.5,project-00


In [21]:
def ensure_data_types(df):
    """Ensure fields adhere to the expected data types."""
    df['user'] = df['user'].astype('str')
    df['hours'] = df['hours'].astype('float32')
    df['project'] = df['project'].astype('str')
    return df

cleaned_df = ensure_data_types(cleaned_df)
cleaned_df.head()

,user,timestamp,hours,project
0,ned,2019-09-27 00:00:00 UTC,8.0,bizdev
1,robert,2019-09-27 00:00:00 UTC,8.0,bizdev
2,ned,2019-09-26 00:00:00 UTC,4.0,bizdev
3,ned,2019-09-26 00:00:00 UTC,1.0,cultureandmanagement
4,ned,2019-09-26 00:00:00 UTC,1.5,project-00


In [22]:
cleaned_df.to_csv('../data/cleaned/cleaned_data.csv', index=False)
cleaned_df.to_pickle('../data/from_scratch/cleaned_data.pkl')

In [23]:
cleaned_df = pd.read_csv('../data/cleaned/cleaned_data.csv')
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19772 entries, 0 to 19771
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user       19772 non-null  object 
 1   timestamp  19772 non-null  object 
 2   hours      19772 non-null  float64
 3   project    19772 non-null  object 
dtypes: float64(1), object(3)
memory usage: 618.0+ KB


In [24]:
df1 = cleaned_df.copy()
df1['id'] = df1['timestamp'].astype(str) + '_' + df1['user'].astype(str)
df1.set_index('id', inplace=True)
df1.head()

,user,timestamp,hours,project
id,,,,
2019-09-27 00:00:00 UTC_ned,ned,2019-09-27 00:00:00 UTC,8.0,bizdev
2019-09-27 00:00:00 UTC_robert,robert,2019-09-27 00:00:00 UTC,8.0,bizdev
2019-09-26 00:00:00 UTC_ned,ned,2019-09-26 00:00:00 UTC,4.0,bizdev
2019-09-26 00:00:00 UTC_ned,ned,2019-09-26 00:00:00 UTC,1.0,cultureandmanagement
2019-09-26 00:00:00 UTC_ned,ned,2019-09-26 00:00:00 UTC,1.5,project-00


In [25]:
df1['hours'] = df1['hours'].astype(str) + ' hr'
df1.head()


,user,timestamp,hours,project
id,,,,
2019-09-27 00:00:00 UTC_ned,ned,2019-09-27 00:00:00 UTC,8.0 hr,bizdev
2019-09-27 00:00:00 UTC_robert,robert,2019-09-27 00:00:00 UTC,8.0 hr,bizdev
2019-09-26 00:00:00 UTC_ned,ned,2019-09-26 00:00:00 UTC,4.0 hr,bizdev
2019-09-26 00:00:00 UTC_ned,ned,2019-09-26 00:00:00 UTC,1.0 hr,cultureandmanagement
2019-09-26 00:00:00 UTC_ned,ned,2019-09-26 00:00:00 UTC,1.5 hr,project-00


In [26]:
df1['user'] = df1['user'].str.title()
df1.head()


,user,timestamp,hours,project
id,,,,
2019-09-27 00:00:00 UTC_ned,Ned,2019-09-27 00:00:00 UTC,8.0 hr,bizdev
2019-09-27 00:00:00 UTC_robert,Robert,2019-09-27 00:00:00 UTC,8.0 hr,bizdev
2019-09-26 00:00:00 UTC_ned,Ned,2019-09-26 00:00:00 UTC,4.0 hr,bizdev
2019-09-26 00:00:00 UTC_ned,Ned,2019-09-26 00:00:00 UTC,1.0 hr,cultureandmanagement
2019-09-26 00:00:00 UTC_ned,Ned,2019-09-26 00:00:00 UTC,1.5 hr,project-00


In [27]:
df2 = cleaned_df.copy()
df2['timestamp'] = pd.to_datetime(df2['timestamp'])
target_date = pd.Timestamp('2018-11-21')
target_week = target_date.isocalendar().week
print(target_week)

47


In [28]:
df2 = df2[df2['timestamp'].dt.isocalendar().week == target_week]
df2.head()

,user,timestamp,hours,project
83,joffrey,2018-11-25 00:00:00+00:00,5.00,project-32
84,theon,2018-11-25 00:00:00+00:00,5.00,events
85,cersei,2018-11-24 08:53:46+00:00,2.00,blogideas
86,hound,2018-11-24 00:00:00+00:00,4.00,finance
87,catelyn,2018-11-23 13:12:16+00:00,0.93,project-32


In [29]:
week_hours = df2.groupby(['user', 'project'])['hours'].sum()
week_hours

user     project             
arya     project-22               1.50
         project-26               3.50
         project-32              33.00
bran     bizdev                  17.87
         cultureandmanagement     1.55
                                 ...  
viserys  learning                 6.95
         opsandadmin              2.19
         project-51              12.13
         project-64               1.97
         project-65               1.29
Name: hours, Length: 212, dtype: float64

In [30]:
df2 = pd.pivot_table(week_hours.reset_index(), values='hours', index='project', columns='user')
df2.head()

user,arya,bran,bronn,catelyn,cersei,daenerys,davos,gendry,hound,jaime,...,robb,robert,samwell,sansa,shae,stannis,theon,tyrion,tywin,viserys
project,,,,,,,,,,,,,,,,,,,,,
airflow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
bizdev,NaN,17.87,1.5,NaN,NaN,NaN,0.50,13.83,NaN,17.5,...,NaN,25.5,NaN,9.0,1.5,NaN,0.5,6.43,NaN,NaN
blog-ideas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
blogideas,NaN,NaN,NaN,NaN,37.0,8.0,0.25,NaN,0.5,NaN,...,NaN,NaN,NaN,NaN,NaN,6.23,13.5,4.53,NaN,12.9
cultureandmanagement,NaN,1.55,NaN,4.97,NaN,1.0,1.00,1.00,NaN,6.0,...,NaN,3.5,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN


In [31]:
df2.fillna(0, inplace=True)
df2.head()

user,arya,bran,bronn,catelyn,cersei,daenerys,davos,gendry,hound,jaime,...,robb,robert,samwell,sansa,shae,stannis,theon,tyrion,tywin,viserys
project,,,,,,,,,,,,,,,,,,,,,
airflow,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.00,0.0,0.00,0.0,0.0
bizdev,0.0,17.87,1.5,0.00,0.0,0.0,0.50,13.83,0.0,17.5,...,0.0,25.5,0.0,9.0,1.5,0.00,0.5,6.43,0.0,0.0
blog-ideas,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0
blogideas,0.0,0.00,0.0,0.00,37.0,8.0,0.25,0.00,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,6.23,13.5,4.53,0.0,12.9
cultureandmanagement,0.0,1.55,0.0,4.97,0.0,1.0,1.00,1.00,0.0,6.0,...,0.0,3.5,0.0,0.0,3.0,0.00,1.0,0.00,0.0,0.0


In [32]:
df3 = cleaned_df.copy()
df3['timestamp'] = pd.to_datetime(df3['timestamp'])
df3 = df3[df3['timestamp'].dt.year == 2017]
df3.head()

,user,timestamp,hours,project
15236,littlefinger,2017-12-31 18:24:42+00:00,7.0,opsandadmin
15237,tywin,2017-12-31 13:24:21+00:00,4.0,workshops
15238,theon,2017-12-30 15:47:07+00:00,8.0,project-40
15239,jorah,2017-12-30 11:27:48+00:00,2.0,project-40
15240,tywin,2017-12-30 00:00:00+00:00,6.0,workshops


In [33]:
df3 = df3.groupby(['project']).agg(total_hours=('hours', 'sum'), user_count=('user', 'nunique'))
df3.head()

,total_hours,user_count
project,,
airflow,9.00,1
bizdev,1041.98,17
blogideas,62.50,6
blogpost,3.50,2
branding,4.50,1


In [34]:
test_dirty_data = pd.read_csv('../data/test/test_dirty_data.csv')

validate_columns_result = validate_columns(test_dirty_data)
validate_hours_result = validate_hours(test_dirty_data)
validate_timestamp_result = validate_timestamp(test_dirty_data)
check_null_values_result = check_null_values(test_dirty_data)

In [35]:
validation_results = []

validation_results.append({"validation_type": "Column Presence", "validation_remarks": validate_columns_result[1]})
validation_results.append({"validation_type": "Hours Validation", "validation_remarks": validate_hours_result[1]})
validation_results.append({"validation_type": "Timestamp Format", "validation_remarks": validate_timestamp_result[1]})
validation_results.append({"validation_type": "Null Values Check", "validation_remarks": check_null_values_result[1]})

validation_report = pd.DataFrame(validation_results)
validation_report.head()

,validation_type,validation_remarks
0,Column Presence,All required columns are present and properly ...
1,Hours Validation,"Invalid 'hours' values in rows: [1, 7, 8, 9, 1..."
2,Timestamp Format,Invalid timestamps: ['Row 0: Invalid type: flo...
3,Null Values Check,"Missing or null values found in rows: [0, 1, 1..."


In [36]:
validation_report.to_csv('../data/reports/output_validation_dirty.csv', index=False)

In [37]:
test_clean_data = pd.read_csv('../data/test/test_clean_data.csv')

validate_columns_result = validate_columns(test_clean_data)
validate_hours_result = validate_hours(test_clean_data)
validate_timestamp_result = validate_timestamp(test_clean_data)
check_null_values_result = check_null_values(test_clean_data)

In [38]:
validation_results = []

validation_results.append({"validation_type": "Column Presence", "validation_remarks": validate_columns_result[1]})
validation_results.append({"validation_type": "Hours Validation", "validation_remarks": validate_hours_result[1]})
validation_results.append({"validation_type": "Timestamp Format", "validation_remarks": validate_timestamp_result[1]})
validation_results.append({"validation_type": "Null Values Check", "validation_remarks": check_null_values_result[1]})

validation_report = pd.DataFrame(validation_results)
validation_report.head()

,validation_type,validation_remarks
0,Column Presence,All required columns are present and properly ...
1,Hours Validation,All 'hours' values are valid
2,Timestamp Format,Invalid timestamps: ['Row 2: Convertible if ch...
3,Null Values Check,"Missing or null values found in rows: [10, 11,..."


In [39]:
validation_report.to_csv('../data/reports/output_validation_cleaned.csv', index=False)

In [40]:
test_clean_data = pd.read_csv('../data/test/test_clean_data.csv')

cleaned_df = remove_null_values(test_clean_data)
cleaned_df = transform_dates(cleaned_df)
cleaned_df = remove_duplicates(cleaned_df)
cleaned_df = strip_whitespace(cleaned_df)
cleaned_df = convert_timestamps(cleaned_df)
cleaned_df = ensure_data_types(cleaned_df)
cleaned_df.head()

,Unnamed: 0,user,timestamp,hours,project
0,3833,stannis,2018-10-02 00:00:00 UTC,0.47,project-10
1,3390,tyrion,2018-10-08 00:00:00 UTC,3.22,learning
2,14661,tormund,2018-02-14 13:08:00 UTC,1.00,learning
3,10007,talisa,2018-07-09 00:00:00 UTC,0.50,opsandadmin
4,15120,littlefinger,2018-01-29 00:00:00 UTC,5.00,opsandadmin


In [41]:
cleaned_df.to_csv('../data/cleaned/output_cleaning.csv', index=False)